<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_10_4_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 10: StreamLit**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 10 Material

Module 10: StreamLit

* Part 10.1: Running StreamLit in Google Colab [[Video]]() [[Notebook]](t81_559_class_10_1_streamlit.ipynb)
* Part 10.2: StreamLit Introduction [[Video]]() [[Notebook]](t81_559_class_10_2_streamlit_intro.ipynb)
* Part 10.3: Understanding Streamlit State [[Video]]() [[Notebook]](t81_559_class_10_3_streamlit_state.ipynb)
* **Part 10.4: Creating a Chat Application** [[Video]]() [[Notebook]](t81_559_class_10_4_chat.ipynb)
* Part 10.5: MultiModal Chat Application [[Video]]() [[Notebook]](t81_559_class_10_5_chat_multimodal.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [17]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai openai streamlit

Note: not using Google CoLab


# Part 10.4: Creating a Chat Application

In this module, we will guide you through the process of creating a StreamLit-based LLM chat application. To keep things accessible and straightforward, we'll run our app using Google Colab. Additionally, we'll introduce you to llm_util.py, a utility script designed to make it easier to work with LangChain-compatible large language models (LLMs). For this example, we'll be using OpenAI's LLM to power our chat application. By the end of this module, you'll have a functional, interactive chat app and a solid understanding of how to integrate LLMs into your projects using StreamLit.

We will now create three files:

* **app.py** - The main StreamLit chat application.
* **llm_util.py** - The LLM utility that allows us to utilize any LangChain LLM for our chat application.
* **llms.yaml** - A config file to define which LLM's we will use; for this example it will be OpenAI.

## Chat Application

This following code sets up a simple chatbot application using the OpenAI API and Streamlit, a Python library that allows for easy creation of web apps. The script, named app.py, begins by importing various modules necessary for its functionality. It uses ```openai.OpenAI``` to interact with the OpenAI language model, streamlit to create the user interface, and sys for handling command-line arguments. Additionally, it imports custom utilities from ```llm_util```, along with specific components from the LangChain library, including ```ConversationSummaryMemory```, ```PromptTemplate```, and ```ConversationChain```. These tools are crucial for managing the conversation's context, creating input templates, and facilitating conversational interactions.

The script starts by checking if the required command-line arguments are provided. It expects a language model profile as an argument; if this is not specified, it halts execution and prompts the user to provide the necessary input. This mechanism ensures that the appropriate language model is selected before launching the chatbot.

The heart of the chatbot setup lies in the create_chatbot function. This function establishes the chatbot using the specified language model and incorporates a memory component, ```ConversationSummaryMemory```, which keeps track of the ongoing conversation's context. By utilizing this memory, the chatbot can generate more coherent and contextually relevant responses over time. Additionally, the function defines a simple template using LangChain's PromptTemplate, formatting how the conversation history and user input are combined. It then returns a ConversationChain object that processes these conversations, ensuring that responses are influenced by the entire chat history.

To create an interactive interface, the script uses Streamlit. It begins by setting a title, "Chat," for the application. To manage the chatbot’s state and keep track of the conversation, the script employs Streamlit's session state mechanism. It checks if a chatbot instance (st.session_state.chat) already exists; if not, it initializes a new one with the LLM profile provided via the command line. Similarly, it sets up a list to store the conversation messages (st.session_state.messages) if it does not already exist.

The conversation history is then iterated through and displayed using Streamlit's chat-specific functions. Each message, whether from the user or the assistant, is rendered in the chat interface. This provides a running log of the conversation, giving the user an ongoing view of their interaction with the chatbot.

User interaction occurs through a chat input box facilitated by st.chat_input. When the user provides input, the script appends the message to the conversation log and displays it in the interface. The chatbot then processes this input using the predict method of ConversationChain, generating a response based on the conversation's context. This response is displayed in the chat window and added to the session state, ensuring the entire dialogue is stored and utilized for subsequent interactions.

In essence, this script leverages LangChain's memory management and templating capabilities to create a dynamic chatbot, while Streamlit provides a user-friendly interface for real-time interaction. The combination of these tools allows the chatbot to maintain context throughout the conversation, resulting in more meaningful and coherent exchanges. Furthermore, by requiring a language model profile as a command-line argument, the script is flexible and adaptable to various LLM configurations, allowing the user to customize the chatbot's behavior depending on their specific needs.

In [18]:
%%writefile app.py
from openai import OpenAI
import streamlit as st
from llm_util import *
from langchain.memory import ConversationSummaryMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
import sys

# This retrieves all command line arguments as a list
arguments = sys.argv
if len(sys.argv) != 2:
    print("Please specify the llm to use as the first argument")
    st.stop()
else:
    profile = sys.argv[1]


def create_chatbot(llm):
    memory = ConversationSummaryMemory(llm=llm)

    template = """{history}\n{input}\n\n
    """
    PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
    return ConversationChain(llm=llm, prompt=PROMPT, memory=memory, verbose=False)


st.title("Chat")

if "chat" not in st.session_state:
    client = open_llm(profile)
    st.session_state.chat = create_chatbot(client)

if "messages" not in st.session_state:
    st.session_state.messages = []

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(
            message["content"],
        )

if prompt := st.chat_input("What is up?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(
            prompt,
        )

    with st.chat_message("assistant"):
        response = st.session_state.chat.predict(input=prompt)
        st.markdown(
            response,
        )
    st.session_state.messages.append({"role": "assistant", "content": response})

Writing app.py


## LLM Utility


This following code enables the chat application to use various language models supported by LangChain based on a configuration file. The llm_util.py script serves as a utility that dynamically loads and initializes different language models using configurations specified in a YAML file (llms.yaml). This approach provides a flexible way to change the language model without modifying the main application code, allowing for easy experimentation and customization.

In [19]:
%%writefile llm_util.py
import yaml

# Load the YAML file
def load_yaml(file_path):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)


# Function to dynamically import a class based on a string path (e.g., "langchain_community.chat_models.ChatOllama")
def get_class(class_path):
    module_path, class_name = class_path.rsplit(".", 1)
    module = __import__(module_path, fromlist=[class_name])
    return getattr(module, class_name)


# Open Language Model Server function
def open_llm(server_name):
    config = load_yaml("llms.yaml")
    for server in config["servers"]:
        if server["name"] == server_name:
            class_path = server["class"]
            clazz = get_class(class_path)
            # Remove 'class' and 'name' from the parameters as they're not needed for initialization
            params = {k: v for k, v in server.items() if k not in ["class", "name"]}

            return clazz(**params)
    raise ValueError(f"Server '{server_name}' not found")

Overwriting llm_util.py


In [20]:
%%writefile llms.yaml

servers:
  - name: server1
    class: langchain_openai.ChatOpenAI
    model: gpt-4o-mini
    temperature: 0



Overwriting llms.yaml


Next, we obtain the password for our StreamLit server we are about to launch.

In [21]:
# !curl https://loca.lt/mytunnelpassword

We launch the StreamLit server and obtain its URL. You will need the above password when you access the URL it gives you.

In [22]:
# !streamlit run app.py server1 &>/content/logs.txt &
# !npx --yes localtunnel --port 8501

The following config file shows how to utilize other LLM servers:

* server1 - Ollama
* server2 - OpenAI
* server3 - Bedrock (AWS)
* server4 - LMStudio

```
servers:
  - name: server1
    class: langchain_community.chat_models.ChatOllama
    base_url: http://localhost:11434
    model: llama2
    temperature: 0
  - name: server2
    class: langchain_openai.ChatOpenAI
    model: gpt-4o-mini
    temperature: 0
  - name: server3
    class: langchain_aws.ChatBedrock
    model_id: amazon.titan-text-express-v1
    model_kwargs:
      temperature: 0.1
  - name: server4
    class: langchain_openai.ChatOpenAI
    base_url: http://localhost:1234/v1/
    model: TheBloke/Mistral-7B-Instruct-v0.1-GGUF
    openai_api_key: None
    temperature: 0
  ```